In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import psycopg2

In [ ]:
# get chrome driver
chrome_service = ChromeService(r'..\drivers\chrome-driver.exe')

In [ ]:
# add options for the driver
options = webdriver.ChromeOptions()

options.add_argument("--start-maximized")

# assign options to the driver
driver = webdriver.Chrome(service=chrome_service, options=options)

In [ ]:
# open web page
driver.get("https://www.transfermarkt.fr/detailsuche/spielerdetail/suche")

In [ ]:
# WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe[id^='sp_message_iframe'][src^='https://cdn.privacy-mgmt.com']")))
# WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@title='ACCEPTER TOUT']"))).click()

iframes = driver.find_elements(By.CSS_SELECTOR, "iframe[id^='sp_message_iframe']")
driver.switch_to.frame(iframes[0])

accept = driver.find_element(By.XPATH, "//button[@title='ACCEPTER TOUT']")
accept.click()

driver.switch_to.default_content()

search = driver.find_element(By.XPATH, "//input[@name='query']")
search.send_keys("ronaldo")
search.send_keys(Keys.ENTER)

In [ ]:
soup = BeautifulSoup(driver.page_source, 'lxml')
titles = soup.find_all("img", class_="bilderrahmen-fixed")

try:
  for title in titles:
    name = title.get("alt")
    image = title.get("src")
  
    connection = psycopg2.connect(user="postgres", password="changeme", host="localhost", port="5432", database="python")
    cursor = connection.cursor()
    cursor.execute('INSERT INTO players (name, image) VALUES (%s,%s)', (name, image))
    connection.commit()
    count = cursor.rowcount

except (Exception, psycopg2.Error) as error:
  print("Failed to insert records", error)
    
except (Exception, psycopg2.IntegrityError) :
  connection.rollback()

finally:
  if connection:
    cursor.close()
    connection.close()  
    
driver.quit()